# Advent of Code 2022
## Day 7
*<https://adventofcode.com/2022/day/7>*

In [1]:
import IPython
import math
import re
from helper import *
from itertools import product, combinations, permutations
from collections import Counter, defaultdict
from string import ascii_lowercase, ascii_uppercase, ascii_letters
from rich import inspect, print, pretty
pretty.install()

In [2]:
DAY = 7
inp = get_input_lines(DAY)
part_1 = part_2 = 0

In [3]:
class File:
    def __init__(self, name, size):
        self.name = name
        self.size = size

    def get_size(self):
        return self.size


class Directory:
    def __init__(self, name, parent):
        self.name = name
        self.parent = parent
        self.files: list[File] = []
        self.directories: list[Directory] = []

    def get_size(self):
        return sum(f.get_size() for f in self.files + self.directories)

    def add_file(self, file: File):
        self.files.append(file)

    def add_directory(self, directory):
        self.directories.append(directory)

    def get_child_dir(self, name: str):
        for d in self.directories:
            if d.name == name:
                return d
        assert False

    def get_all_subdirs(self):
        yield self
        for d in self.directories:
            yield from d.get_all_subdirs()

    def get_smallest_greater_than(self, size) -> int:
        return min(
            [self.get_size()] + [d.get_smallest_greater_than(size) for d in self.directories],
            key=lambda n: n if n > size else math.inf,
        )


In [4]:
root = Directory("/", None)
cwd = root

i = 0
while i < len(inp):
    assert cwd is not None
    line = inp[i]
    parts = line.split()
    if line.startswith("$"):
        if parts[1] == "cd":
            if parts[2] == "/":
                cwd = root
            elif parts[2] == "..":
                cwd = cwd.parent
            else:
                cwd = cwd.get_child_dir(parts[2])
            i += 1

        elif parts[1] == "ls":
            i += 1
            while i < len(inp) and not inp[i].startswith("$"):
                line = inp[i]
                parts = line.split()

                if parts[0] == "dir":
                    cwd.add_directory(Directory(parts[1], cwd))
                else:
                    cwd.add_file(File(parts[1], int(parts[0])))
                i += 1

In [5]:
part_1 = sum(s for d in root.get_all_subdirs() if (s := d.get_size()) <= 100000)

In [6]:
def get_unused_space(dir):
    return 70000000 - dir.get_size()


required_space = 30000000 - get_unused_space(root)
part_2 = root.get_smallest_greater_than(required_space)


In [7]:
print_part_1(part_1)
print_part_2(part_2)

Part 1: 1648397

Part 2: 1815525

In [8]:
# submit_part_1(part_1, DAY)
# submit_part_2(part_2, DAY)